In [ ]:
# === Fase 3 - Criação da Tabela SGV_OCORRENCIA_SAC via SQLite + Pandas ===

import sqlite3
import pandas as pd
import json

# Conectar ao banco de dados
conn = sqlite3.connect('../data/melhores_compras.db')

# Carregar tabelas
sac = pd.read_sql_query("SELECT * FROM sac", conn)
produto = pd.read_sql_query("SELECT * FROM produto", conn)
cliente = pd.read_sql_query("SELECT * FROM cliente", conn)
funcionario = pd.read_sql_query("SELECT * FROM funcionario", conn)

# Carregar margens de lucro e ICMS do JSON
with open("../requisitos/1_5_arquivo_produto.json", "r", encoding="utf-8") as f:
    dados_json = json.load(f)

df_lucro = pd.DataFrame(dados_json['produtos'])
df_lucro['nome'] = df_lucro['nome'].str.strip()

# Merge produto + lucro
produto_full = produto.merge(df_lucro[['nome', 'icms']], on='nome', how='left')
produto_full['vl_perc_lucro'] = 20  # Valor arbitrário fixo
produto_full['vl_unitario_lucro_produto'] = (produto_full['vl_perc_lucro'] / 100) * produto_full['preco']

# Merge SAC + Cliente + Produto + Funcionário
base = sac \
    .merge(cliente, on='id_cliente') \
    .merge(produto_full, on='id_produto') \
    .merge(funcionario, on='id_funcionario')

# Mapear tipo de chamado
tipo_map = {"1": "SUGESTÃO", "2": "RECLAMAÇÃO"}
base['ds_tipo_classificacao_sac'] = base['tipo_chamado'].map(tipo_map)
base['vl_icms_produto'] = None

# Selecionar colunas para nova tabela
# Atualizado:
ocorrencia = base[[
    'id_sac', 'data_hora_atendimento', 'ds_tipo_classificacao_sac', 'id_produto', 'nome_x', 'preco',
    'vl_perc_lucro', 'vl_unitario_lucro_produto', 'id_cliente', 'nome_y', 'id_funcionario', 'nome', 'vl_icms_produto'
]].rename(columns={
    'id_sac': 'nr_ocorrencia_sac',
    'data_hora_atendimento': 'dt_abertura_sac',
    'nome_x': 'ds_produto',
    'preco': 'vl_unitario_produto',
    'nome_y': 'nm_cliente',
    'nome': 'nm_funcionario'
})


# Salvar no banco de dados
ocorrencia.to_sql("sgv_ocorrencia_sac", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

# Visualizar
print("Tabela sgv_ocorrencia_sac criada com sucesso. Exemplo de dados:")
display(ocorrencia.head())


Tabela sgv_ocorrencia_sac criada com sucesso. Exemplo de dados:


,nr_ocorrencia_sac,dt_abertura_sac,ds_tipo_classificacao_sac,id_produto,ds_produto,vl_unitario_produto,vl_perc_lucro,vl_unitario_lucro_produto,id_cliente,nm_cliente,id_funcionario,nm_funcionario,vl_icms_produto
0,1,None,SUGESTÃO,1,Alice Silva,1999.9,20,399.98,1,Smartphone XYZ,1,Marcos Lima,None
1,2,None,RECLAMAÇÃO,1,Bruno Souza,1999.9,20,399.98,2,Smartphone XYZ,2,Fernanda Rocha,None
2,3,None,SUGESTÃO,1,Carla Mendes,1999.9,20,399.98,3,Smartphone XYZ,1,Marcos Lima,None


In [4]:
ocorrencia = base[[
    'id_sac', 'data_hora_atendimento', 'ds_tipo_classificacao_sac',
    'id_produto', 'nome_x', 'preco', 'vl_perc_lucro', 'vl_unitario_lucro_produto',
    'id_cliente', 'nome_y', 'id_funcionario', 'nome', 'vl_icms_produto'
]].rename(columns={
    'id_sac': 'nr_ocorrencia_sac',
    'data_hora_atendimento': 'dt_abertura_sac',
    'nome_x': 'ds_produto',     # nome do produto
    'preco': 'vl_unitario_produto',
    'nome_y': 'nm_cliente',     # nome do cliente
    'nome': 'nm_funcionario'    # nome do funcionário
})


In [5]:
ocorrencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   nr_ocorrencia_sac          3 non-null      int64  
 1   dt_abertura_sac            0 non-null      object 
 2   ds_tipo_classificacao_sac  3 non-null      object 
 3   id_produto                 3 non-null      int64  
 4   ds_produto                 3 non-null      object 
 5   vl_unitario_produto        3 non-null      float64
 6   vl_perc_lucro              3 non-null      int64  
 7   vl_unitario_lucro_produto  3 non-null      float64
 8   id_cliente                 3 non-null      int64  
 9   nm_cliente                 3 non-null      object 
 10  id_funcionario             3 non-null      int64  
 11  nm_funcionario             3 non-null      object 
 12  vl_icms_produto            0 non-null      object 
dtypes: float64(2), int64(5), object(6)
memory usage: 444.0

## Problemas atuais para resolver:

### Coluna id_categoria duplicada - teste existência da coluna antes de tentar criá-la.

### Tabela de resultado vazia (categorias sem chamados) - a tabela sgv_ocorrencia_sac não foi atualizada após alterações na tabela produto.

In [1]:
import sqlite3
import pandas as pd

# 1. Conectar ao banco
conn = sqlite3.connect("../data/melhores_compras.db")

# 2. Ler as tabelas necessárias
sac = pd.read_sql_query("SELECT * FROM sac", conn)
produto = pd.read_sql_query("SELECT * FROM produto", conn)
cliente = pd.read_sql_query("SELECT * FROM cliente", conn)
funcionario = pd.read_sql_query("SELECT * FROM funcionario", conn)

# 3. Merge atualizado (com id_categoria)
base = sac \
    .merge(cliente, on='id_cliente') \
    .merge(produto, on='id_produto') \
    .merge(funcionario, on='id_funcionario')

# 4. Mapear tipo_chamado
tipo_dict = {"1": "SUGESTÃO", "2": "RECLAMAÇÃO"}
base['ds_tipo_classificacao_sac'] = base['tipo_chamado'].map(tipo_dict)
base['vl_icms_produto'] = None

# 5. Selecionar colunas para nova tabela sgv_ocorrencia_sac
ocorrencia = base[[
    'id_sac', 'data_hora_atendimento', 'ds_tipo_classificacao_sac', 'id_produto', 'nome_x', 'preco',
    'vl_icms_produto', 'id_cliente', 'nome_y', 'id_funcionario', 'nome'
]].rename(columns={
    'id_sac': 'nr_ocorrencia_sac',
    'data_hora_atendimento': 'dt_abertura_sac',
    'nome_x': 'ds_produto',
    'preco': 'vl_unitario_produto',
    'nome_y': 'nm_cliente',
    'nome': 'nm_funcionario'
})

# 6. Salvar a tabela atualizada no banco
ocorrencia.to_sql("sgv_ocorrencia_sac", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print("Tabela sgv_ocorrencia_sac atualizada com sucesso.")


Tabela sgv_ocorrencia_sac atualizada com sucesso.
